In [3]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dropout,Dense,ReLU,Softmax

In [4]:
df = pd.read_csv('E:/Documents/GitHub/Express-U/data/preprocessed/canny_labels.csv',index_col=False)
df

,file_paths,labels
0,E:/Documents/GitHub/Express-U/data/preprocesse...,1
1,E:/Documents/GitHub/Express-U/data/preprocesse...,1
2,E:/Documents/GitHub/Express-U/data/preprocesse...,1
3,E:/Documents/GitHub/Express-U/data/preprocesse...,1
4,E:/Documents/GitHub/Express-U/data/preprocesse...,1
...,...,...
41995,E:/Documents/GitHub/Express-U/data/preprocesse...,Z
41996,E:/Documents/GitHub/Express-U/data/preprocesse...,Z
41997,E:/Documents/GitHub/Express-U/data/preprocesse...,Z
41998,E:/Documents/GitHub/Express-U/data/preprocesse...,Z


In [5]:
X_Train,X_Test = train_test_split(df,train_size=0.8,random_state=42,shuffle=True)

In [6]:
Train_image_generator = ImageDataGenerator(rescale=1/255,
                                        zoom_range=0.5,
                                        brightness_range=[0.6,1.0],
                                        rotation_range=20,
                                        validation_split=0.1)
Validation_image_generator = ImageDataGenerator(rescale=1/255)

In [7]:
Train_Set = Train_image_generator.flow_from_dataframe(dataframe=X_Train,
                                                   x_col="file_paths",
                                                   y_col="labels",
                                                   batch_size=32,
                                                   class_mode="categorical",
                                                   color_mode="grayscale",
                                                   subset="training")

Validation_Set = Validation_image_generator.flow_from_dataframe(dataframe=X_Test,
                                                   x_col="file_paths",
                                                   y_col="labels",
                                                   batch_size=32,
                                                   class_mode="categorical",
                                                   color_mode="grayscale",
                                                   shuffle=False)

Found 30240 validated image filenames belonging to 35 classes.
Found 8400 validated image filenames belonging to 35 classes.


In [8]:
Train_Set.image_shape

(256, 256, 1)

In [9]:
inputshape = (256,256,1)
outputshape = 35

model = Sequential()
model.add(Conv2D(input_shape=inputshape,filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(64,activation="relu"))
model.add(Dense(outputshape,activation="softmax"))


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 64)      640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 524288)            0

In [11]:
lr=0.001
model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=lr), metrics=['accuracy'])

In [12]:
history = model.fit(Train_Set,epochs=10,steps_per_epoch=(Train_Set.image_shape[0] // 32),validation_steps=(Validation_Set.image_shape[0] // 32),batch_size=32,verbose=1,validation_data=Validation_Set)

Epoch 1/10
8/8 [==============================] - 713s 90s/step - loss: 3.5145 - accuracy: 0.0586 - val_loss: 3.3267 - val_accuracy: 0.1484
Epoch 2/10
8/8 [==============================] - 481s 56s/step - loss: 3.3002 - accuracy: 0.1484 - val_loss: 2.6192 - val_accuracy: 0.2344
Epoch 3/10
8/8 [==============================] - 341s 44s/step - loss: 3.0793 - accuracy: 0.2188 - val_loss: 2.2863 - val_accuracy: 0.4023
Epoch 4/10
8/8 [==============================] - 307s 39s/step - loss: 2.6590 - accuracy: 0.3281 - val_loss: 1.4886 - val_accuracy: 0.6367
Epoch 5/10
8/8 [==============================] - 290s 37s/step - loss: 2.1465 - accuracy: 0.4492 - val_loss: 0.8651 - val_accuracy: 0.7188
Epoch 6/10
8/8 [==============================] - 250s 32s/step - loss: 1.8735 - accuracy: 0.5078 - val_loss: 0.7095 - val_accuracy: 0.7891
Epoch 7/10
8/8 [==============================] - 254s 32s/step - loss: 1.4582 - accuracy: 0.6016 - val_loss: 0.2782 - val_accuracy: 0.9414
Epoch 8/10
8/8 [====

In [13]:
history.history

{'loss': [3.514519453048706,
  3.3002443313598633,
  3.079265594482422,
  2.659022808074951,
  2.1464507579803467,
  1.873488426208496,
  1.4582101106643677,
  1.1524150371551514,
  0.9536014199256897,
  0.9856067299842834],
 'accuracy': [0.05859375,
  0.1484375,
  0.21875,
  0.328125,
  0.44921875,
  0.5078125,
  0.6015625,
  0.671875,
  0.74609375,
  0.72265625],
 'val_loss': [3.32668137550354,
  2.6191675662994385,
  2.286271095275879,
  1.4885789155960083,
  0.8651072978973389,
  0.7095006704330444,
  0.2782427966594696,
  0.17775210738182068,
  0.20452380180358887,
  0.19975844025611877],
 'val_accuracy': [0.1484375,
  0.234375,
  0.40234375,
  0.63671875,
  0.71875,
  0.7890625,
  0.94140625,
  0.95703125,
  0.94921875,
  0.9375]}